In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from xgboost import XGBClassifier
from sklearn.utils import class_weight
from joblib import dump, load
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

In [2]:
# choose whether to filter (EDIT maybe)
filterNaive = True
filterGenes = True

# load data
adata = sc.read(r'all_samples_GE_clusters.h5ad')
print ('loaded data')

# filter out NaNs
adata = adata[adata.obs['specificity'].notnull()]
print('filtered out NaNs')

# filter out Naive cells
if (filterNaive) {
    adata = adata[adata.obs['cluster'] != 'Naive']
    print('filtered out Naive cells')
}

# filter out TCR/BCR/ribosomal/MT genes
if (filterGenes) {
    tcr = adata.var['Gene_Name'].str.contains(r'^TR[ABGD][VDJC]')
    bcr = adata.var['Gene_Name'].str.contains(r'^IG[KGLH][JVCMGAED]')
    rib = adata.var['Gene_Name'].str.contains(r'^RP[SL][[:digit:]]|^RPLP[[:digit:]]|^RPSA')
    mt = adata.var['Gene_Name'].str.contains(r'^MT-')
    adata = adata[:,(~tcr & ~bcr & ~rib & ~mt)]
    print('filtered out TCR, BCR, Ribosomal, and MT genes')
}

# labels known MCPyV vs not
X = pd.DataFrame(adata.X, columns=adata.var.index.values)

Y = adata.obs['specificity']
Y = pd.Series(Y, dtype="category")
Y_known = (Y == 'MCPyV')
print('labeled known MCPyV vs not')

# split into test and train
# random_state to set random seed
xtrain, xtest, ytrain, ytest=train_test_split(X, Y_known, test_size=0.15, random_state=0)
print('split into test and train')


loaded data
filtered out NaNs
labeled known MCPyV vs not
split into test and train


In [ ]:
# IF YOU WANT TO RETRAIN A MODEL:
# best params determined by RandomizedSearchCV
xgb_model = XGBClassifier(n_estimators=1000, min_child_weight=4, max_depth=3, gamma=0.1, eta=0.35, colsample_bytree=0.6)

# fit model to 
print('training model...')
xgb_model.fit(xtrain, ytrain)
print('model trained')
print()

scores = cross_val_score(xgb_model, xtrain, ytrain, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())

# if you want to save the model (EDIT maybe whatever name you want with extension .joblib)
# can later load the model
dump(xgb_model, 'CITN09_RF.joblib')

In [3]:
# IF YOU WANT TO LOAD A PREVIOUS MODEL: EDIT maybe (if you want a different model file)
xgb_model = load('CITN09_RF_all_random0.joblib')

In [4]:
# predict on test data
y_pred = xgb_model.predict(xtest)
print(classification_report(ytest, y_pred))

              precision    recall  f1-score   support

       False       0.94      0.99      0.96      2365
        True       0.70      0.29      0.41       227

    accuracy                           0.93      2592
   macro avg       0.82      0.64      0.68      2592
weighted avg       0.91      0.93      0.91      2592



In [5]:
# get top features
results=pd.DataFrame()
results['GeneID']=xtest.columns
results['importances'] = xgb_model.feature_importances_
results.sort_values(by='importances',ascending=False,inplace=True)

top_features = results[:20]
top_features

,GeneID,importances
473,ENSG00000275791,0.030501
789,ENSG00000211810,0.017548
421,ENSG00000158321,0.015327
667,ENSG00000149428,0.013816
1062,ENSG00000167633,0.013553
451,ENSG00000237702,0.011337
288,ENSG00000113088,0.010122
980,ENSG00000176890,0.009298
468,ENSG00000211727,0.008550
229,ENSG00000173193,0.007304


In [6]:
# get info on top features from adata.var
df_top_features = adata.var.loc[top_features['GeneID']]
df_top_features

,Gene_Name,Info,n_cells,highly_variable,means,dispersions,dispersions_norm
GeneID,,,,,,,
ENSG00000275791,TRBV10-3,Gene Expression,640,True,0.074775,2.279632,4.248985
ENSG00000211810,TRAV29DV5,Gene Expression,903,True,0.035318,1.058022,1.465935
ENSG00000158321,AUTS2,Gene Expression,3373,True,0.100272,0.684261,0.614440
ENSG00000149428,HYOU1,Gene Expression,2160,True,0.056497,0.636291,0.505156
ENSG00000167633,KIR3DL1,Gene Expression,969,True,0.042605,1.075438,1.505612
ENSG00000237702,TRBV3-1,Gene Expression,1446,True,0.128783,1.955578,3.510730
ENSG00000113088,GZMK,Gene Expression,9303,True,0.523412,1.662351,2.063091
ENSG00000176890,TYMS,Gene Expression,495,True,0.022706,1.211338,1.815217
ENSG00000211727,TRBV7-6,Gene Expression,1095,True,0.117614,2.086128,3.808149


In [7]:
df_top_features['Gene_Name']

GeneID
ENSG00000275791      TRBV10-3
ENSG00000211810     TRAV29DV5
ENSG00000158321         AUTS2
ENSG00000149428         HYOU1
ENSG00000167633       KIR3DL1
ENSG00000237702       TRBV3-1
ENSG00000113088          GZMK
ENSG00000176890          TYMS
ENSG00000211727       TRBV7-6
ENSG00000173193        PARP14
ENSG00000111796         KLRB1
ENSG00000103187         COTL1
ENSG00000173585          CCR9
ENSG00000196126      HLA-DRB1
ENSG00000107331         ABCA2
ENSG00000226380    AC016831.1
ENSG00000278030       TRBV7-9
ENSG00000211753        TRBV28
ENSG00000181827          RFX7
ENSG00000211814        TRAV35
Name: Gene_Name, dtype: category
Categories (1202, object): ['A2M-AS1', 'ABCA1', 'ABCA2', 'ABCA7', ..., 'ZNF683', 'ZNF777', 'ZYX', 'ZZEF1']

In [ ]:
# save top features in csv if you want
df_top_features.to_csv('top_features_info_all_random0.csv')